In [1]:
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import activations

import os
import math
import cv2
import numpy as np
import pandas as pd
import tensorflow as tf

In [2]:
def get_sequential_model(input_shape):
    model = keras.Sequential(
        [
            # input
            layers.Input(input_shape),
            
            # 1번쨰 layer
            layers.Conv2D(64,3,strides = 1,activation = "relu",padding = 'same'), # 이미지를 위한 레이어
            layers.Conv2D(64,3,strides = 1,activation = "relu",padding = 'same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.5),
            
            # 2번쨰 layer
            layers.Conv2D(128,3,strides = 1,activation = "relu",padding = 'same'),
            layers.Conv2D(128,3,strides = 1,activation = "relu",padding = 'same'),
            layers.MaxPool2D(),
            layers.BatchNormalization(),
            layers.Dropout(0.3),
            
            # Classifier
            layers.GlobalMaxPool2D(),
            layers.Dense(128,activation = "relu"),
            layers.Dense(1,activation = "sigmoid")
        ]
    )
    return model

In [3]:
input_shape = (256,256,3)
model = get_sequential_model(input_shape)

model.compile(
    optimizer = 'adam',
    loss = 'binary_crossentropy',
    metrics = 'accuracy'
)

model.summary() # 모델의 성능 확인

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 256, 256, 64)      1792      
                                                                 
 conv2d_1 (Conv2D)           (None, 256, 256, 64)      36928     
                                                                 
 max_pooling2d (MaxPooling2D  (None, 128, 128, 64)     0         
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 128, 128, 64)     256       
 ormalization)                                                   
                                                                 
 dropout (Dropout)           (None, 128, 128, 64)      0         
                                                                 
 conv2d_2 (Conv2D)           (None, 128, 128, 128)     7

In [4]:
class DataGenerator(keras.utils.Sequence):
    def __init__(self, batch_size, csv_path, fold, image_size, mode='train', shuffle=True):
        self.batch_size = batch_size
        self.fold = fold
        self.image_size = image_size
        self.mode = mode
        self.shuffle = shuffle
        
        self.df = pd.read_csv(csv_path)
        
        if self.mode == 'train':
            self.df = self.df[self.df['fold'] != self.fold]
        elif self.mode == 'val':
            self.df = self.df[self.df['fold'] == self.fold]
        
        self.on_epoch_end()
        
        #### https://github.com/tensorflow/models/issues/3134
        #### 파일 이슈 -> 삭제
        invalid_filenames = [
            'Egyptian_Mau_14',
            'Egyptian_Mau_139',
            'Egyptian_Mau_145',
            'Egyptian_Mau_156',
            'Egyptian_Mau_167',
            'Egyptian_Mau_177',
            'Egyptian_Mau_186',
            'Egyptian_Mau_191',
            'Abyssinian_5',
            'Abyssinian_34',
            'chihuahua_121',
            'beagle_116'
        ]
        self.df = self.df[~self.df['file_name'].isin(invalid_filenames)]
        
        self.on_epoch_end()

        
    
    def on_epoch_end(self):
        if self.shuffle:
            self.df = self.df.sample(frac=1).reset_index(drop=True)

    # len()
    def __len__(self):
        return math.ceil(len(self.df) / self.batch_size)

    def __getitem__(self, idx):
        strt = idx * self.batch_size
        fin = (idx + 1) * self.batch_size
        data = self.df.iloc[strt:fin]
        batch_x, batch_y = self.get_data(data)
        return np.array(batch_x), np.array(batch_y)
        
    def get_data(self, data):
        batch_x = []
        batch_y = []
        
        for _, r in data.iterrows():
            file_name = r['file_name']
            image = cv2.imread(f'data/images/{file_name}.jpg')
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = cv2.resize(image, (self.image_size, self.image_size))
            image = image / 255.
        
            label = int(r['species']) - 1
            
            batch_x.append(image)
            batch_y.append(label)
        return batch_x, batch_y

In [5]:
csv_path = 'data/kfolds.csv'

train_generator = DataGenerator(
    batch_size = 128,
    csv_path = csv_path,
    fold = 1, 
    image_size = 256,
    mode = 'train',
    shuffle = True
)

valid_generator = DataGenerator(
    batch_size = 128,
    csv_path = csv_path,
    fold = 1, 
    image_size = 256,
    mode = 'val',
    shuffle = True
)

In [6]:
# Early Stopping
# 무조건 epoch을 많이 돌린 후, 특정 시점에 멈추게 하는 기능
# monitor : EarlyStopping의 기준이 되는 값
# val_loss는 val_loss 값이 더이상 감소되지 않을 경우 EarlyStopping을 적용
# patience: Training이 진행됨에도 더이상 monitor되는 값의 개선이 없을 경우, 몇 번의 epoch을 진행할 지 정하는 값
# mode: monitor되는 값이 최소가 되야 하는지, 최대가 되어야 하는지 설정하는 값
# restore_best_weights: true라면 training이 끝난 후, model의 weight를 monitor하고 있던 값이 가장 좋았을 경우 weight로 복원,
# False라면 마지막 training이 끝난 후 weight로 설정

early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor = 'val_loss',patience = 3,verbose = 1,mode = 'min', restore_best_weights = False
)

In [7]:
# ReduceLROnPlateau
# monitor: ReduceLROnPlateau의 기준이 되는 값, val_loss가 더이상 감소되지 않을 경우 ReduceLROnPlateau를 적용
# factor: learning rate를 얼마나 감소시킬지 정하는 값, 새로운 learning rate = learning rate * factor
# patience : Training이 진행됨에도 더이상 monitor되는 값의 개선이 없을 경우, 최적의 monitor 값을 기준으로
# learning rate를 조절할지의 값을 설정함

reduce_on_plateau = tf.keras.callbacks.ReduceLROnPlateau(
    monitor = 'val_loss', factor = 0.1, patience = 10, verbose = 1, model = 'min', min_lr = 0.001
)

In [8]:
# 모델 경로 
# 모델 경로를 '{epoch:02d}-{val_loss:.2f}.hdf5'라고 하면, 앞에 명시한 문자열로 파일을 저장함
# 예: 01-0.12.h5
# save_weights_only: True -> weights만 저장, False -> 모델, layer, weights 모두 저장
filepath = '{epoch:02d}-{val_loss:.2f}.hdf5'

model_checkpoint = tf.keras.callbacks.ModelCheckpoint(
    filepath, monitor = 'val_loss', verbose=1, save_best_only=True,
    save_weights_only=False, mode='min'
)

In [9]:
history = model.fit(
    train_generator,
    validation_data = valid_generator,
    epochs = 10,
    callbacks = [
        early_stopping,
        reduce_on_plateau,
        model_checkpoint
        
    ],
    verbose = 1
)

Epoch 1/10
46/46 [==============================] - ETA: 0s - loss: 0.9414 - accuracy: 0.6433 
Epoch 1: val_loss improved from inf to 0.65266, saving model to 01-0.65.hdf5
46/46 [==============================] - 973s 21s/step - loss: 0.9414 - accuracy: 0.6433 - val_loss: 0.6527 - val_accuracy: 0.6767 - lr: 0.0010
Epoch 2/10
46/46 [==============================] - ETA: 0s - loss: 0.6309 - accuracy: 0.6699 
Epoch 2: val_loss did not improve from 0.65266
46/46 [==============================] - 970s 21s/step - loss: 0.6309 - accuracy: 0.6699 - val_loss: 0.6854 - val_accuracy: 0.5950 - lr: 0.0010
Epoch 3/10
46/46 [==============================] - ETA: 0s - loss: 0.6108 - accuracy: 0.6730 
Epoch 3: val_loss did not improve from 0.65266
46/46 [==============================] - 971s 21s/step - loss: 0.6108 - accuracy: 0.6730 - val_loss: 0.6597 - val_accuracy: 0.6481 - lr: 0.0010
Epoch 4/10
46/46 [==============================] - ETA: 0s - loss: 0.5990 - accuracy: 0.6837 
Epoch 4: val_loss